# 00 — Basis Machine Learning: Supervised vs Unsupervised

Dit notebook is bedoeld als **zachte landing** vóór neural networks.

We behandelen in eenvoudige taal:
1. Supervised vs unsupervised learning
2. K-Means (unsupervised)
3. K-Nearest Neighbours (supervised)
4. Waarom en hoe je data splitst in **train / validation / test**

Doelgroep: niet-technische deelnemers die in 1 dag een goed gevoel willen krijgen voor AI-modellen.

## Supervised vs Unsupervised (telecom-context)

**Supervised learning** = je hebt historische voorbeelden met het juiste antwoord (label).  
Voorbeeld telecom: je weet van oude klanten wie is opgezegd (**churn**) en leert een model om churn te voorspellen.

**Unsupervised learning** = je hebt geen labels, maar zoekt patronen in de data.  
Voorbeeld telecom: je groepeert klanten in segmenten op basis van gebruiksgedrag, zonder dat iemand vooraf labels heeft gegeven.

| Type | Input | Output | Voorbeeld |
|---|---|---|---|
| Supervised | Data + labels | Voorspelling | Churn ja/nee |
| Unsupervised | Data zonder labels | Groepen/patronen | Klantsegmenten |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

np.random.seed(42)

## Deel A — Unsupervised learning met K-Means

We maken kunstmatige telecomdata met twee kenmerken:
- `gemiddelde_belminuten`
- `datagebruik_gb`

In de praktijk heb je vaak veel meer kenmerken, maar dit helpt om het visueel te begrijpen.

In [ ]:
n_klanten = 240

segment_1 = np.column_stack([
    np.random.normal(80, 12, n_klanten // 3),
    np.random.normal(4, 1.0, n_klanten // 3)
])
segment_2 = np.column_stack([
    np.random.normal(220, 25, n_klanten // 3),
    np.random.normal(10, 1.5, n_klanten // 3)
])
segment_3 = np.column_stack([
    np.random.normal(420, 40, n_klanten // 3),
    np.random.normal(24, 2.5, n_klanten // 3)
])

X_unsup = np.vstack([segment_1, segment_2, segment_3])
telecom_unsup = pd.DataFrame(X_unsup, columns=['gemiddelde_belminuten', 'datagebruik_gb'])
telecom_unsup.head()

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(telecom_unsup)
centra = kmeans.cluster_centers_

plt.figure(figsize=(8, 6))
plt.scatter(telecom_unsup['gemiddelde_belminuten'], telecom_unsup['datagebruik_gb'], c=clusters, cmap='viridis', alpha=0.7)
plt.scatter(centra[:, 0], centra[:, 1], c='red', marker='X', s=250, label='Cluster-centrum')
plt.title('K-Means klantsegmentatie (zonder labels)')
plt.xlabel('Gemiddelde belminuten')
plt.ylabel('Datagebruik (GB)')
plt.legend()
plt.show()

telecom_unsup['cluster'] = clusters
telecom_unsup.groupby('cluster')[['gemiddelde_belminuten', 'datagebruik_gb']].mean().round(1)

## Deel B — Supervised learning met KNN

Nu gebruiken we de echte churn-dataset. Hier **hebben** we labels (`Churn`), dus dit is supervised learning.

We gaan extra aandacht geven aan **train / validation / test split**.

In [ ]:
df = pd.read_csv('../data/churn-bigml-20.csv')
df['Churn'] = df['Churn'].astype(int)

categorical_cols = df.select_dtypes(include='object').columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

X = df_encoded.drop(columns='Churn')
y = df_encoded['Churn']

# Eerst train (60%) en tijdelijk (40%)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.40, random_state=42, stratify=y
)

# Daarna tijdelijk splitsen in validation (20%) en test (20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42, stratify=y_temp
)

print(f'Train: {len(X_train)} rijen')
print(f'Validation: {len(X_val)} rijen')
print(f'Test: {len(X_test)} rijen')

In [ ]:
split_namen = ['Train', 'Validation', 'Test']
split_groottes = [len(X_train), len(X_val), len(X_test)]

plt.figure(figsize=(7, 4))
bars = plt.bar(split_namen, split_groottes, color=['steelblue', 'orange', 'seagreen'])
plt.title('Visuele verdeling van train/validation/test')
plt.ylabel('Aantal rijen')

for b in bars:
    plt.text(b.get_x() + b.get_width()/2, b.get_height()+3, int(b.get_height()), ha='center')

plt.show()

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

k_waarden = list(range(1, 21))
val_scores = []

for k in k_waarden:
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train_scaled, y_train)
    val_pred = model.predict(X_val_scaled)
    val_scores.append(accuracy_score(y_val, val_pred))

beste_k = k_waarden[int(np.argmax(val_scores))]
print(f'Beste k op validation-set: {beste_k}')

plt.figure(figsize=(8, 4))
plt.plot(k_waarden, val_scores, marker='o')
plt.title('K kiezen met validation-score')
plt.xlabel('k (aantal buren)')
plt.ylabel('Accuracy op validation-set')
plt.grid(alpha=0.3)
plt.show()

In [ ]:
beste_model = KNeighborsClassifier(n_neighbors=beste_k)
beste_model.fit(X_train_scaled, y_train)
test_pred = beste_model.predict(X_test_scaled)

print(f'Test accuracy: {accuracy_score(y_test, test_pred):.3f}')
print()
print('Classificatierapport:')
print(classification_report(y_test, test_pred, target_names=['Geen churn', 'Wel churn']))

cm = confusion_matrix(y_test, test_pred)
plt.figure(figsize=(5, 4))
plt.imshow(cm, cmap='Blues')
plt.title('Confusion matrix (test-set)')
plt.xlabel('Voorspeld')
plt.ylabel('Werkelijk')
plt.xticks([0, 1], ['Geen churn', 'Wel churn'])
plt.yticks([0, 1], ['Geen churn', 'Wel churn'])

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha='center', va='center')

plt.colorbar()
plt.tight_layout()
plt.show()

### Uitleg Confusion Matrix (niet-technisch)

Denk aan de matrix als een scorebord met 4 vakjes:

- **Linksboven (True Negative)**: model zegt **geen churn** en dat klopt.
- **Rechtsboven (False Positive)**: model zegt **wel churn**, maar klant zou blijven (vals alarm).
- **Linksonder (False Negative)**: model zegt **geen churn**, maar klant vertrekt toch (gemiste churner).
- **Rechtsonder (True Positive)**: model zegt **wel churn** en dat klopt.

Praktisch voor telecom:

- Veel **false negatives** = je mist klanten die vertrekken.
- Veel **false positives** = je benadert te veel klanten onnodig.
- Vaak wil je een balans, of juist extra focus op het verminderen van gemiste churners.

## Samenvatting

- **K-Means** helpt je groepen te ontdekken zonder labels (unsupervised).
- **KNN** voorspelt labels op basis van gelijkenis met bekende voorbeelden (supervised).
- De **train/validation/test split** voorkomt dat je jezelf voor de gek houdt met te optimistische resultaten.

In het volgende notebook maken we de stap naar **logistische regressie** als brug naar neural nets.